**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using Theano backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [50]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

The function act has two parameters, ```s``` and ```train```. ```s``` is current state, and next act will be proposed according to the policy of the agent as well as the current state. 

However, the policy is optimized gradually based on the previous (s,a,r)-sequence. At the beginning, policy is not optimal. If we always choose next action according to the policy, policy optimization issue is likely to fall into local optimum. That is the reason why ```epsilon``` is essential. If ```train``` is true, an additional random exploration of epsilon is allowed. That means the next action can be chosen randomly from the whole action space with a probability of ```epsilon```, which provides a wider training set and more information for policy optimization.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:      # going down
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:    # going up
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:   # going right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:   # going left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [53]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=80 # set small when debugging
epochs_test=40 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position```: an array of size (grid_size, grid_size) that marks the position of the rat as 1 as well as the boundry of board as -1. The rest of the values in this array are 0. Because the rat has a visibility of 2 cells, increasing the side length of the board by two fake cells can simplify the structure of ```state```. In ```position```, the values of outermost two-circle increased cells are -1. 


```board```: an array of size (grid_size, grid_size) that marks the positions of cheese as 0.5 and poisonous cells as -1. The rest of the values in this array are 0. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [38]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [39]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return score/epochs

In [40]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.0/12.0. Average score (-7.0)
Win/lose count 10.5/14.0. Average score (-5.25)
Win/lose count 10.0/6.0. Average score (-2.1666666666666665)
Win/lose count 10.0/5.0. Average score (-0.375)
Win/lose count 12.5/16.0. Average score (-1.0)
Win/lose count 10.0/14.0. Average score (-1.5)
Win/lose count 6.5/23.0. Average score (-3.642857142857143)
Win/lose count 6.5/19.0. Average score (-4.75)
Win/lose count 7.0/23.0. Average score (-6.0)
Win/lose count 10.0/10.0. Average score (-5.4)
Win/lose count 10.0/13.0. Average score (-5.181818181818182)
Win/lose count 8.5/16.0. Average score (-5.375)
Win/lose count 6.5/10.0. Average score (-5.230769230769231)
Win/lose count 13.5/14.0. Average score (-4.892857142857143)
Win/lose count 10.5/15.0. Average score (-4.866666666666666)
Win/lose count 7.5/14.0. Average score (-4.96875)
Win/lose count 8.5/13.0. Average score (-4.9411764705882355)
Win/lose count 6.0/9.0. Average score (-4.833333333333333)
Win/lose count 13.0/9.0. Average score (-4

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Q5-Answer__:

(1)
\begin{align}
Q^\pi(s,a)&=E[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=E[r(s,a)+\sum_{1\leq t \leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
&=E[r(s,a)+\sum_{s',a'}\sum_{0\leq t \leq T}(\gamma^{t+1}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a')p(s_{1}=s',a_{1}=a'|s_{0}=s,a_{0}=a)]\\
&=E[\sum_{s',a'}p(s_{1}=s',a_{1}=a'|s_{0}=s,a_{0}=a)r(s,a)+\sum_{s',a'}p(s_{1}=s',a_{1}=a'|s_{0}=s,a_{0}=a)\gamma \sum_{0\leq t \leq T}(\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a')]\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

(2)
\begin{align}
Q^{\pi}(s,a)&=r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]\\
&=r(s,a) + \gamma \sum_{s'}p(S_{t+1}=s' | S_t=s,a_t=a)\sum_{a'}\pi(a'|s')[Q^{\pi}(s',a')]\\
\end{align}

Let

\begin{equation}
\pi^{*}(a|s) = \left\{
\begin{array}{rcl}
1       &      & if \quad a = argmax_a Q^{*}(s,a)\\
0    &      & otherwise\\
\end{array} \right.
\end{equation}
so 
\begin{align}
Q^{*}(s,a)&=r(s,a) + \gamma \sum_{s'}p(S_{t+1}=s' | S_t=s,a_t=a)max_{a'}[Q^{*}(s',a')]\\
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align}

(3)
In q-learning, the target of q value is $r+\gamma max_{a}Q(S_{t+1},a)$. Therefore, we could use this here as the loss function.
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [66]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [67]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [43]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,5,5,self.n_state]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            m = self.memory.random_access()
            target_q[i] =self.model.predict(m[0].reshape([1,5,5,self.n_state]))[0]
            if m[4]:
                ######## FILL IN
                input_states[i] = m[0]
                target_q[i,m[2]] = m[3]
            else:
                ######## FILL IN
                input_states[i] = m[0]
                target_q[i,m[2]] = m[3] + self.discount * np.max(self.model.predict(m[1].reshape([1,5,5,self.n_state]))[0])
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(50,activation ='tanh'))
#         model.add(BatchNormalization())
        model.add(Dense(20,activation ='relu'))
#         model.add(BatchNormalization())
        model.add(Dense(4))
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [44]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/080 | Loss 0.009302499704062939 | Win/lose count 1.0/3.0 (-2.0)
Epoch 001/080 | Loss 0.005043152254074812 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/080 | Loss 0.00676990719512105 | Win/lose count 4.0/7.0 (-3.0)
Epoch 003/080 | Loss 0.018927041441202164 | Win/lose count 3.0/1.0 (2.0)
Epoch 004/080 | Loss 0.01218363642692566 | Win/lose count 1.0/3.0 (-2.0)
Epoch 005/080 | Loss 0.056992799043655396 | Win/lose count 2.5/4.0 (-1.5)
Epoch 006/080 | Loss 0.012360656633973122 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/080 | Loss 0.06137796863913536 | Win/lose count 5.5/4.0 (1.5)
Epoch 008/080 | Loss 0.004770689643919468 | Win/lose count 2.0/3.0 (-1.0)
Epoch 009/080 | Loss 0.004848767071962357 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/080 | Loss 0.003746370319277048 | Win/lose count 1.5/1.0 (0.5)
Epoch 011/080 | Loss 0.002492089755833149 | Win/lose count 2.5/3.0 (-0.5)
Epoch 012/080 | Loss 0.0036356006748974323 | Win/lose count 4.5/1.0 (3.5)
Epoch 013/080 | Loss 0.002457749331369996 | Win

I tested three architectures and several parameters:

| Architectures | Details |
|:----:|:----:|
|  1  | 1 hidden layer with 30 neurons |
|  2  | 2 hidden layers with 50 and 20 neurons |
|  3  | 2 hidden layers with BatchNormalization |

Then the results are shown in the table below (epochs_train=40, epochs_test=20):

| Architectures | Learning rate | Batch size   | Memory size |  Average Score(20 epochs)  |
|:----:|:----:|:----:|:----:|:----:|
|  1  |  0.1  | 32 |  2000  | -0.1 |
|  __2__  |  __0.1__  | __32__ |  __2000__  | __-0.025__ |
|  3  |  0.1  | 32 |  2000  | -0.225 |
|  1  |  0.3  | 32 |  2000  | -0.25  |
|  2  |  0.3  | 32 |  2000  | -0.275 |
|  3  |  0.3  | 32 |  2000  | -0.225 |
|  1  |  0.1  | 16 |  2000  | -0.225 |
|  2  |  0.1  | 16 |  2000  | -0.125 |
|  3  |  0.1  | 16 |  2000  | -0.3 |
|  1  |  0.3  | 16 |  2000  | -0.325 |
|  2  |  0.3  | 16 |  2000  | -0.15 |
|  3  |  0.3  | 16 |  2000  | -0.35 |
|  1  |  0.1  | 32 |  4000  | -0.25 |
|  2  |  0.1  | 32 |  4000  | -0.45 |
|  3  |  0.1  | 32 |  4000  | -0.25 |
|  1  |  0.3  | 32 |  4000  | -0.125 |
|  2  |  0.3  | 32 |  4000  | -0.275 |
|  3  |  0.3  | 32 |  4000  | -0.475 |
|  1  |  0.1  | 16 |  4000  | -0.175 |
|  2  |  0.1  | 16 |  4000  | -0.35 |
|  3  |  0.1  | 16 |  4000  | -0.3 |
|  1  |  0.3  | 16 |  4000  | -0.4 |
|  2  |  0.3  | 16 |  4000  | -0.35 |
|  3  |  0.3  | 16 |  4000  | -0.475 |

These results are average over only 20 epochs, thus there is an inevitable stochasticity. However we can still find that:
* Architecture 2 has a better performance in 5 cases (8 in total) and it is a bad idea to use batchnormalization in this task; 
* Learning rate 0.1 works better in 9/12 cases
* Batch size 32 is better in 8/12 cases
* Memory size 2000 is better in 9/12 cases

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [46]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='tanh'))
        model.add(Conv2D(20,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
      
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [47]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/080 | Loss 0.008316217921674252 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/080 | Loss 0.002900427207350731 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/080 | Loss 0.0077551016584038734 | Win/lose count 2.0/9.0 (-7.0)
Epoch 003/080 | Loss 0.0031828884966671467 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/080 | Loss 0.009943469427525997 | Win/lose count 5.5/2.0 (3.5)
Epoch 005/080 | Loss 0.004795393440872431 | Win/lose count 3.5/0 (3.5)
Epoch 006/080 | Loss 0.004530586302280426 | Win/lose count 1.5/0 (1.5)
Epoch 007/080 | Loss 0.04960665479302406 | Win/lose count 5.5/4.0 (1.5)
Epoch 008/080 | Loss 0.002791428007185459 | Win/lose count 7.5/3.0 (4.5)
Epoch 009/080 | Loss 0.0019096765900030732 | Win/lose count 5.0/7.0 (-2.0)
Epoch 010/080 | Loss 0.002471021842211485 | Win/lose count 10.0/2.0 (8.0)
Epoch 011/080 | Loss 0.001928087673150003 | Win/lose count 14.5/1.0 (13.5)
Epoch 012/080 | Loss 0.004192729946225882 | Win/lose count 7.0/3.0 (4.0)
Epoch 013/080 | Loss 0.004504343029111624 | Win

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [69]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.5/4.0. Average score (8.5)
Win/lose count 10.0/0. Average score (9.25)
Win/lose count 7.5/1.0. Average score (8.333333333333334)
Win/lose count 5.0/3.0. Average score (6.75)
Win/lose count 17.0/1.0. Average score (8.6)
Win/lose count 6.0/1.0. Average score (8.0)
Win/lose count 14.5/4.0. Average score (8.357142857142858)
Win/lose count 2.5/4.0. Average score (7.125)
Win/lose count 10.5/0. Average score (7.5)
Win/lose count 3.0/0. Average score (7.05)
Win/lose count 15.5/1.0. Average score (7.7272727272727275)
Win/lose count 13.5/2.0. Average score (8.041666666666666)
Win/lose count 12.0/2.0. Average score (8.192307692307692)
Win/lose count 12.0/1.0. Average score (8.392857142857142)
Win/lose count 5.5/1.0. Average score (8.133333333333333)
Win/lose count 12.5/3.0. Average score (8.21875)
Win/lose count 8.5/1.0. Average score (8.176470588235293)
Win/lose count 11.0/2.0. Average score (8.222222222222221)
Win/lose count 12.5/2.0. Average score (8.342105263

-2.425

In [34]:
HTML(display_videos('cnn_test10.mp4'))

In [35]:
HTML(display_videos('fc_test10.mp4'))


| Temperature | Final scores with FC | Final scores with CNN |
|:----:|:----:|:----:|
|  0.1  | -0.3925 | 1.685 |
|  0.2  | -0.125 | 2.57 |
|  0.3  | -0.225 | 3.855 |
|  0.4  | 0.105 | 4.7125 |
|  0.5  | -0.35 | 5.39 |
|  0.6  | 0.505 | 7.525 |
|  0.7  | 1.01 | 9.283 |
|  0.8  | 1.64 | 10.725 |
|  0.9  | 2.22 | 14.698 |


With the change of temperature, the average final scores over 10 epochs are listed in the above table. We can see that the performance of CNN is widely better than FC. In particular, when temperature is large, the scores of FC change linearly while when temperature is small, the incertainty of FC results increases. This may be because when the density of bonus or malus is small, the reward of action is almost likely be 0, which does not provide helps for improvement. In this case, the exploration is not enough and the result is more random.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [104]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    decay_rate = 0.1

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_rate))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:      # going down
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:    # going up
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:   # going right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:   # going left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.3
        reward = reward + self.board[self.x, self.y]
        
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [105]:
# Training
epochs_train = 40
epochs_test=20
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/040 | Loss 0.0233 | Win/lose count 13.0/50.89999999999988 (-37.89999999999988)
Epoch 001/040 | Loss 0.0253 | Win/lose count 8.5/50.09999999999988 (-41.59999999999988)
Epoch 002/040 | Loss 0.0139 | Win/lose count 21.5/33.900000000000034 (-12.400000000000034)
Epoch 003/040 | Loss 0.0173 | Win/lose count 28.5/32.60000000000004 (-4.100000000000037)
Epoch 004/040 | Loss 0.0444 | Win/lose count 23.5/31.10000000000004 (-7.6000000000000405)
Epoch 005/040 | Loss 0.0316 | Win/lose count 19.0/25.50000000000004 (-6.500000000000039)
Epoch 006/040 | Loss 0.0425 | Win/lose count 18.5/32.90000000000004 (-14.400000000000041)
Epoch 007/040 | Loss 0.0211 | Win/lose count 20.0/32.90000000000004 (-12.900000000000041)
Epoch 008/040 | Loss 0.0212 | Win/lose count 20.0/38.099999999999994 (-18.099999999999994)
Epoch 009/040 | Loss 0.0211 | Win/lose count 12.5/42.79999999999995 (-30.299999999999947)
Epoch 010/040 | Loss 0.0428 | Win/lose count 13.5/41.79999999999996 (-28.29999999999996)
Epoch 011/040 

In [107]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 13.5/7.0. Average score (6.5)
Win/lose count 6.5/1.0. Average score (6.0)
Win/lose count 19.5/5.0. Average score (8.833333333333334)
Win/lose count 16.0/6.0. Average score (9.125)
Win/lose count 21.0/11.0. Average score (9.3)
Win/lose count 19.0/1.0. Average score (10.75)
Win/lose count 21.0/10.0. Average score (10.785714285714286)
Win/lose count 23.0/7.0. Average score (11.4375)
Win/lose count 11.5/4.0. Average score (11.0)
Win/lose count 15.5/6.0. Average score (10.85)
Win/lose count 21.5/5.0. Average score (11.363636363636363)
Win/lose count 10.5/3.0. Average score (11.041666666666666)
Win/lose count 24.5/4.0. Average score (11.76923076923077)
Win/lose count 6.5/2.0. Average score (11.25)
Win/lose count 18.0/11.0. Average score (10.966666666666667)
Win/lose count 7.0/1.0. Average score (10.65625)
Win/lose count 14.5/4.0. Average score (10.647058823529411)
Win/lose count 19.0/3.0. Average score (10.944444444444445)
Win/lose count 15.0/2.0. Average score (11.05263157894

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [108]:
class DQN_mimick(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_mimick, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,5,5,self.n_state]))[0])

    def reinforce(self, s_,a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_,a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            # do not need to learn, just mimick what the expert did
            s_, a_= self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 3  # a large fixed value
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_CNN_mimick(DQN_mimick):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN_mimick, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(50,activation ='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [109]:
def train_mimick(expert, agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = expert.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [110]:
# Training
epochs_train = 60
epochs_test = 20
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_mimick = DQN_CNN_mimick(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_mimick(agent, agent_mimick, env, epochs_train, prefix='cnn_train_mimick')
HTML(display_videos('cnn_train_mimick10.mp4'))

Epoch 000/060 | Loss 0.4071 | Win/lose count 14.0/9.0 (5.0)
Epoch 001/060 | Loss 0.3066 | Win/lose count 15.5/4.0 (11.5)
Epoch 002/060 | Loss 0.4727 | Win/lose count 11.0/2.0 (9.0)
Epoch 003/060 | Loss 0.2413 | Win/lose count 17.0/3.0 (14.0)
Epoch 004/060 | Loss 0.1884 | Win/lose count 27.0/9.0 (18.0)
Epoch 005/060 | Loss 0.3713 | Win/lose count 18.0/9.0 (9.0)
Epoch 006/060 | Loss 0.4620 | Win/lose count 20.0/7.0 (13.0)
Epoch 007/060 | Loss 0.4444 | Win/lose count 11.0/2.0 (9.0)
Epoch 008/060 | Loss 0.2427 | Win/lose count 11.5/2.0 (9.5)
Epoch 009/060 | Loss 0.3185 | Win/lose count 10.5/2.0 (8.5)
Epoch 010/060 | Loss 0.4169 | Win/lose count 11.5/5.0 (6.5)
Epoch 011/060 | Loss 0.2354 | Win/lose count 19.5/5.0 (14.5)
Epoch 012/060 | Loss 0.2817 | Win/lose count 20.5/8.0 (12.5)
Epoch 013/060 | Loss 0.2615 | Win/lose count 19.0/3.0 (16.0)
Epoch 014/060 | Loss 0.3817 | Win/lose count 12.5/4.0 (8.5)
Epoch 015/060 | Loss 0.3324 | Win/lose count 14.5/4.0 (10.5)
Epoch 016/060 | Loss 0.2713 | Wi

In [112]:
# Evaluation
test(agent_mimick,env,epochs_test,prefix='cnn_test_mimick')
HTML(display_videos('cnn_test_mimick10.mp4'))

Win/lose count 9.0/1.0. Average score (8.0)
Win/lose count 1.5/2.0. Average score (3.75)
Win/lose count 8.5/2.0. Average score (4.666666666666667)
Win/lose count 15.0/5.0. Average score (6.0)
Win/lose count 5.5/3.0. Average score (5.3)
Win/lose count 8.5/3.0. Average score (5.333333333333333)
Win/lose count 11.5/6.0. Average score (5.357142857142857)
Win/lose count 15.0/1.0. Average score (6.4375)
Win/lose count 10.0/4.0. Average score (6.388888888888889)
Win/lose count 8.0/1.0. Average score (6.45)
Win/lose count 10.5/4.0. Average score (6.454545454545454)
Win/lose count 11.5/0. Average score (6.875)
Win/lose count 15.0/3.0. Average score (7.269230769230769)
Win/lose count 6.0/0. Average score (7.178571428571429)
Win/lose count 22.5/8.0. Average score (7.666666666666667)
Win/lose count 11.5/4.0. Average score (7.65625)
Win/lose count 9.0/0. Average score (7.735294117647059)
Win/lose count 8.5/2.0. Average score (7.666666666666667)
Win/lose count 4.0/0. Average score (7.473684210526316

We can see that, when temperature is 0.3, without exploration, the average score is around 3.8, while that of expert(DQN_CNN_exploration) is 11.3 and the result of DQN_mimic is 7.7. DQN_mimic only tries to mimick the expert's behavior, therefore it can not have a better performance than expert. However if it mimicks great, it will work better than DQN without exploration.

***